In [ ]:
%matplotlib inline
import tensorflow as tf
#tf.keras.mixed_precision.Policy('mixed_float16')
import glob
import numpy as np
import cv2
from tensorflow.keras.layers import Dense, Input, Conv2D, Conv2DTranspose, Flatten, MaxPooling2D, Add, LeakyReLU, \
                                    Reshape, Dropout, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
initializer = tf.keras.initializers.RandomNormal(stddev=0.02)
from IPython.display import clear_output

In [ ]:
def imshow(image):
    image = np.float32((image + 1) / 2)
    plt.imshow(image[..., ::-1])

In [ ]:
def load_images(path, size=128, count=float('inf')):
    paths = glob.glob(path+'/*.*')
    
    images = []
    
    for n, i in enumerate(paths):
        if n == count:
            break
        image = cv2.imread(i)
        if len(image) >= size and len(image[0]) >= size:
            images.append(cv2.resize(image, (size, size)))
    images = np.uint8(images)
    return images

In [ ]:
class DataGenerator:
    def __init__(self, path, size=128, batch_size=8, count=float('inf')):
        self.images = load_images(path, size, count)
        self.batch_size = batch_size
    def get_random_samples(self):
        indexes = np.random.randint(0, len(self.images), size=self.batch_size)
        
        ret = np.float16(self.images[indexes]) / 127.5 - 1
        
        return ret

In [ ]:
datagen = DataGenerator('./animefaces256cleaner')

In [ ]:
imshow(datagen.get_random_samples()[0])

In [ ]:
def build_generator():
    input_layer = Input(shape=(128,))
    
    ## PREPARE
    dense_1 = Dense(512*8*8, kernel_initializer=initializer)(input_layer)
    dense_1 = BatchNormalization()(dense_1)
    dense_1 = ReLU()(dense_1)
    
    reshape = Reshape((8, 8, 512))(dense_1)
    
    conv_1 = Conv2DTranspose(256, 3, padding='same', strides=2, kernel_initializer=initializer)(reshape)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = LeakyReLU(0.2)(conv_1)
    
    conv_1 = Conv2DTranspose(128, 3, padding='same', strides=2, kernel_initializer=initializer)(conv_1)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = LeakyReLU(0.2)(conv_1)
    
    conv_1 = Conv2DTranspose(128, 3, padding='same', strides=1, kernel_initializer=initializer)(conv_1)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = LeakyReLU(0.2)(conv_1)
    
    conv_1 = Conv2DTranspose(64, 3, padding='same', strides=2, kernel_initializer=initializer)(conv_1)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = LeakyReLU(0.2)(conv_1)
    
    conv_1 = Conv2DTranspose(32, 3, padding='same', strides=2, kernel_initializer=initializer)(conv_1)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = LeakyReLU(0.2)(conv_1)
    
    final = Conv2D(3, 9, padding='same', activation='tanh')(conv_1)

    return Model(input_layer, final)

In [ ]:
def build_discriminator():
    input_layer = Input(shape=(128, 128, 3), dtype=tf.float16)
    
    c1 = Conv2D(20, 4, padding='same', strides=1, kernel_initializer=initializer)(input_layer)
    c1 = LeakyReLU(0.2)(c1)
    
    c2 = Conv2D(40, 4, padding='same', strides=2, kernel_initializer=initializer)(c1)
    c2 = LeakyReLU(0.2)(c2)
    
    c2 = Conv2D(80, 4, padding='same', strides=2, kernel_initializer=initializer)(c2)
    c2 = LeakyReLU(0.2)(c2)
    
    c2 = Conv2D(160, 4, padding='same', strides=2, kernel_initializer=initializer)(c2)
    c2 = LeakyReLU(0.2)(c2)
    
    c2 = Conv2D(320, 4, padding='same', strides=2, kernel_initializer=initializer)(c2)
    c2 = LeakyReLU(0.2)(c2)

    flatten = Flatten()(c2)
    
    final = Dense(1)(flatten)

    return Model(input_layer, final)

In [ ]:
gen = build_generator()
disc = build_discriminator()

discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)

In [ ]:
disc.summary()

In [ ]:
tf.keras.utils.plot_model(disc, show_shapes=True)

In [ ]:
image = datagen.get_random_samples()[0]# + np.random.normal(size=(64, 64, 3), loc=0, scale=0.00)

imshow(image)

In [ ]:
imshow(gen(np.random.normal(size=(1, 128)), training=True)[0])

In [ ]:
imshow(gen.predict(np.random.normal(size=(1, 128)))[0])

In [ ]:
disc.predict(datagen.get_random_samples())

In [ ]:
print(disc.predict(np.random.normal(size=(1, 128, 128, 3)))[0])

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(prediction_fake):
    ones = tf.ones_like(prediction_fake)
    return loss(ones, prediction_fake)

def discriminator_loss(prediction_fake, prediction_real):
    ones = tf.random.uniform(minval=0.9, maxval=0.93, shape=prediction_real.shape)
    #zeros = tf.random.uniform(minval=0, maxval=0.5, shape=prediction_fake.shape)
    
    #ones = tf.ones_like(prediction_real)
    zeros = tf.zeros_like(prediction_fake)
    
    return loss(ones, prediction_real) + \
           loss(zeros, prediction_fake)

In [ ]:
BATCH_SIZE = 8
@tf.function
def train_step(real_images):
    noise = tf.random.normal(shape=(BATCH_SIZE, 128))
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_images = gen(noise, training=True)
    
        fake_desicions = disc(fake_images, training=True)
        real_desicions = disc(real_images, training=True)
    
        gen_loss = generator_loss(fake_desicions)
        disc_loss = discriminator_loss(fake_desicions, real_desicions)
    
    
    generator_gradients = gen_tape.gradient(gen_loss, gen.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss, disc.trainable_variables)
    
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, disc.trainable_variables))
    generator_optimizer.apply_gradients(zip(generator_gradients, gen.trainable_variables))
    
    return gen_loss, disc_loss        

In [ ]:
@tf.function
def generate_samle():
    pred = gen(tf.random.normal(shape=(1, 128)))[0]
    pred = (pred + 1) / 2 * 255
    return pred

In [ ]:
for i in range(0, 500_000):
    batch = datagen.get_random_samples()
    
    try:
        gl, dl = train_step(batch)
    except KeyboardInterrupt:
        break
    gl = round(float(gl.numpy()), 4)
    dl = round(float(dl.numpy()), 4)
    print(f'\rbatch: {i}\tgl: {gl}\tdl: {dl}\t\r')
    
    if (i % 200) == 0:
        cv2.imwrite(f'samples/{i}.png', np.uint8(generate_samle()))
        clear_output(wait=True)
    
    if (i + 1) % 300 == 0:
        generator_optimizer.learning_rate = generator_optimizer.learning_rate * 0.995
        discriminator_optimizer.learning_rate = discriminator_optimizer.learning_rate * 0.995

In [ ]:
discriminator_optimizer.learning_rate

In [ ]:
a = np.sum(np.float64(images), axis=0) / len(images)
imshow(a) # mean of all images

In [ ]:
gen.save('gen128_best2.h5')
disc.save('disc128_best2.h5')